# Convert labels from .xml to .txt

In order to augment our images we first need to convert .xml files into .txt files. Here is the code to do that.

In [ ]:
# Convert to txt file for augmentation

import xml.etree.ElementTree as ET
import os
xml_dir = 'C:/Users/kscho/Desktop/plate_xml/'#path to xml file folder
xmls = os.listdir(xml_dir)
all_labels = []
for xml in xmls:
    line = ''
    image_name = xml.split('.')[0]+'.jpeg'
    tree = ET.parse(xml_dir+xml)
    line = 'C:/Users/kscho/Desktop/plate_img/'+image_name+' '  # path to image file folder
    root = tree.getroot()
    for objct in root.iter('object'):
        node = objct.find('bndbox')
        xmin = node.find('xmin').text
        xmax = node.find('xmax').text
        ymin = node.find('ymin').text
        ymax = node.find('ymax').text
        obj_name = objct.find('name').text
        if (obj_name=='plate'):
            line += xmin +","+ ymin +","+ xmax+","+ ymax+" "+obj_name+' '
            
    all_labels.append(line)

with open('C:/Users/kscho/Desktop/labels.txt','w') as f:   #generated txt file
    for line in all_labels:
        f.write(line+'\n')

# Open labels.txt

Checking the content of .txt file. The number of files should be equal to the number of xml files.

In [ ]:
# printing total number of files

import os
with open('C:/Users/kscho/Desktop/labels.txt','r') as f:
    contents = f.readlines()
len(contents)

# Making directory for augmented images

1. Make folder **(in this case aug_img)** to store augmented images.

In [ ]:
image_dir = 'C:/Users/kscho/Desktop/plate_img/'#IMAGE DIR
images = os.listdir(image_dir)
save_dir = 'C:/Users/kscho/Desktop/aug_img/'#AUGMENT IMAGES DIR
labels_txt = 'C:/Users/kscho/Desktop/labels.txt'#ANNOTATION TEXT DIR
output_file= 'C:/Users/kscho/Desktop/augment.txt'#AUGMENT TEXT
len(images)

# Augmenting Images and saving xml files.

## Steps 

1. change the index number in **image_name = line.split(" ")[0].split("/")[5]** according to your directory structure.
2. Assign unique name to augmented images in **f.write("C:/Users/kscho/Desktop/aug_img/"+'p7'+image_name+" "+cont+'\n').**
3. This will also create augment.txt. This will be used in next step to produce .xml files for augmented images.

In [ ]:
# Augmentation 

import imgaug as ia
from imgaug import augmenters as iaa

import cv2
import os, shutil

ia.seed(1)

for line in contents:
    line = line.replace("\n","")
    image_name = line.split(" ")[0].split("/")[5]
    print(image_name)#image name
    image = cv2.imread(image_dir+image_name)
    b_boxes = line.split(" ")[1:2]#Bounding box coordinates
    obj=line.split(" ")[2]#object name
    li_class = []
    li_ia = []
    for box in b_boxes:
        box = box.split(",")
        xmin, ymin, xmax, ymax = float(box[0]), float(box[1]), float(box[2]), float(box[3])
        li_ia.append(ia.BoundingBox(x1=xmin, y1=ymin, x2=xmax, y2=ymax))
    bbs = ia.BoundingBoxesOnImage(li_ia, shape=image.shape)
    seq = iaa.Sequential(
    [
#        iaa.Fliplr(1),
#        iaa.Affine(rotate=20)
#        iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
#        iaa.Multiply((0.5, 1.5), per_channel=0.5),
#        iaa.Add((-10, 10), per_channel=0.5),
#        iaa.Affine(translate_px={"x": 40, "y": 60}, rotate=(-0.5,2.75))
        iaa.Flipud(0.2)

    ])
    seq_det = seq.to_deterministic()
    image_aug = seq_det.augment_images([image])[0]
    bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]
    
    for i in range(len(bbs.bounding_boxes)):
        cont = ""
        
        before = bbs.bounding_boxes[i]
        after = bbs_aug.bounding_boxes[i]
        cont += str(int(after.x1)) +","+ str(int(after.y1)) +","+ \
                  str(int(after.x2)) +","+ str(int(after.y2))+" "+str(obj)
        print(image_name)

        print("BB %d: (%.4f, %.4f, %.4f, %.4f) -> (%.4f, %.4f, %.4f, %.4f)" % (
            i,
            before.x1, before.y1, before.x2, before.y2,
            after.x1, after.y1, after.x2, after.y2)
        )
        print(cont)
        
    with open(output_file,"a") as f:
        f.write("C:/Users/kscho/Desktop/aug_img/"+'p7'+image_name+" "+cont+'\n')#TEXT

    cv2.imwrite(save_dir+'p7'+image_name,image_aug)#IMAGES

# Creating augmented .xml file

## Steps

1. Change **src_dir = "C:/Users/kscho/Desktop/aug_img/"** add path for folder in which have generated augmented images in above step.
2. Change the index in this line **image_name = splitted[0].split("/")[5].**
3. Change path in this line **ET.SubElement(root, "path").text = 'K:/Data/fashion/aug_img/'+image_name.** Add path of your augmented image folder.
4. Change path in **tree.write("C:/Users/kscho/Desktop/aug_xml/"+file_name+".xml").** This will store .xml files for augmented images.

**NOTE** Every time you create new .xml files delete the contents in **augmented.txt**

In [ ]:
labels = ! cat C:/Users/kscho/Desktop/augment.txt

In [ ]:
# Converting from txt to xml

src_dir = "C:/Users/kscho/Desktop/aug_img/"
folder_name = "allimages"
for img in labels:
    splitted = img.split(" ")
    image_name = splitted[0].split("/")[5]#image name
    x=img.split(" ")[2]#object name
    boxes = splitted[1:-1]
    
    print(src_dir+image_name)
    image = cv2.imread(src_dir+image_name)
    print(image_name)
    image_info = image.shape
    print(image_info)
    height, width, depth = image_info[0], image_info[1], image_info[2]
    root = ET.Element("annotation")
    ET.SubElement(root, "folder").text = folder_name
    ET.SubElement(root, "filename").text = image_name
    ET.SubElement(root, "path").text = 'K:/Data/fashion/aug_img/'+image_name
    ET.SubElement(ET.SubElement(root, "source"), "database").text = "Unknown"
    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(width)
    ET.SubElement(size, "height").text = str(height)
    ET.SubElement(size, "depth").text = str(depth)
    ET.SubElement(root, "segmented").text = '0'

    for box in boxes:
        box = box.split(',')
        
        obj_name = x
        xmin = str(int(float(box[0])))
        ymin = str(int(float(box[1])))
        xmax = str(int(float(box[2])))
        ymax = str(int(float(box[3])))
        obj = ET.SubElement(root, "object")
        ET.SubElement(obj, "name").text = obj_name
        ET.SubElement(obj, "pose").text = "Unspecified"
        ET.SubElement(obj, "truncated").text = "0"
        ET.SubElement(obj, "difficult").text = "0"
        bndbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bndbox, "xmin").text = xmin
        ET.SubElement(bndbox, "ymin").text = ymin
        ET.SubElement(bndbox, "xmax").text = xmax
        ET.SubElement(bndbox, "ymax").text = ymax
    tree = ET.ElementTree(root)
    file_name = image_name.split('.')[0]
    tree.write("C:/Users/kscho/Desktop/aug_xml/"+file_name+".xml")
    file_name = None